In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('fer2013.csv')

In [3]:
print(dataset)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [5]:
width, height = 48, 48
datapoints = dataset['pixels'].tolist()
X = []
for xseq in datapoints:
    xvals = [int(xp) for xp in xseq.split(' ')]
    xvals = np.asarray(xvals).reshape(width, height)
    X.append(xvals.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

In [6]:
y = pd.get_dummies(dataset['emotion']).to_numpy()

#storing them using numpy
np.save('dataX', X)
np.save('labels', y)

print("Number of Features in dataset: "+str(len(X[0])))
print("Number of Labels in dataset: "+ str(len(y[0])))
print("Number of examples in dataset:"+str(len(X)))
print("X stored as dataX.npy and y stored as labels.npy ")

Number of Features in dataset: 48
Number of Labels in dataset: 7
Number of examples in dataset:35887
X stored as dataX.npy and y stored as labels.npy 


In [7]:
import sys, os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

In [8]:
num_features = 48
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

x = np.load('./dataX.npy')
y = np.load('./labels.npy')

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

#splitting into training, validation and testing data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

#saving the test samples to be used later
np.save('Xtest', X_test)
np.save('ytest', y_test)

In [9]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

2021-12-11 00:46:13.164171: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 48)        20784     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 48)        192       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 48)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 96)        41568     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 96)        3

In [10]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

#training the model
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_valid), np.array(y_valid)),
          shuffle=True)

#saving the  model to be used later
emotions_json = model.to_json()
with open("emotions.json", "w") as json_file:
    json_file.write(emotions_json)
model.save_weights("emotionCNN.h5")
print("Saved model")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-12-11 00:46:19.420619: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
455/455 [==============================] - 758s 2s/step - loss: 1.9747 - accuracy: 0.2190 - val_loss: 1.8134 - val_accuracy: 0.2594
Epoch 2/100
455/455 [==============================] - 729s 2s/step - loss: 1.8442 - accuracy: 0.2429 - val_loss: 1.8032 - val_accuracy: 0.2607
Epoch 3/100
455/455 [==============================] - 676s 1s/step - loss: 1.8145 - accuracy: 0.2522 - val_loss: 1.7440 - val_accuracy: 0.3000
Epoch 4/100
455/455 [==============================] - 678s 1s/step - loss: 1.7447 - accuracy: 0.2864 - val_loss: 1.6297 - val_accuracy: 0.3248
Epoch 5/100
455/455 [==============================] - 685s 2s/step - loss: 1.6522 - accuracy: 0.3326 - val_loss: 1.5041 - val_accuracy: 0.4115
Epoch 6/100
455/455 [==============================] - 679s 1s/step - loss: 1.5567 - accuracy: 0.3802 - val_loss: 1.4658 - val_accuracy: 0.4046
Epoch 7/100
455/455 [==============================] - 672s 1s/step - loss: 1.4929 - accuracy: 0.4095 - val_loss: 1.3727 - val_accuracy:

455/455 [==============================] - 673s 1s/step - loss: 0.7261 - accuracy: 0.7445 - val_loss: 1.0326 - val_accuracy: 0.6536
Epoch 58/100
455/455 [==============================] - 676s 1s/step - loss: 0.7287 - accuracy: 0.7401 - val_loss: 1.0224 - val_accuracy: 0.6579
Epoch 59/100
455/455 [==============================] - 689s 2s/step - loss: 0.7044 - accuracy: 0.7502 - val_loss: 1.0568 - val_accuracy: 0.6523
Epoch 60/100
455/455 [==============================] - 686s 2s/step - loss: 0.7060 - accuracy: 0.7543 - val_loss: 1.0352 - val_accuracy: 0.6625
Epoch 61/100
455/455 [==============================] - 676s 1s/step - loss: 0.6960 - accuracy: 0.7579 - val_loss: 1.0835 - val_accuracy: 0.6585
Epoch 62/100
455/455 [==============================] - 678s 1s/step - loss: 0.6855 - accuracy: 0.7621 - val_loss: 1.1059 - val_accuracy: 0.6567
Epoch 63/100
455/455 [==============================] - 672s 1s/step - loss: 0.6858 - accuracy: 0.7624 - val_loss: 1.0452 - val_accuracy: 0.657

In [12]:
filenames = os.listdir("/Users/riley/Documents/Python Documents/ArtificialIntelligence/SemesterProject/faces")
test_df = pd.DataFrame({'filename' : filenames})    
samples = test_df.shape[0]

In [15]:
from tensorflow import keras

test_data = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_data.flow_from_dataframe(
    test_df, 
    "/Users/riley/Documents/Python Documents/ArtificialIntelligence/SemesterProject/faces", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=[48,488],
    batch_size=batch_size,
    shuffle=False)

Found 3 validated image filenames.


In [24]:
predict = model.predict_generator(test_generator, steps=np.ceil(samples/batch_size), input_shape = (height, width, 1))

TypeError: predict_generator() got an unexpected keyword argument 'input_shape'